# DispNet

import (tensorflow, numpy) package <br>
numpy use to make numpy array batch data for training <br>
tensorflow use to make FlowNet structure

In [ ]:
import tensorflow as tf
import numpy as np

dataset : Scene Flow Datasets <br>
model : DispNet structure <br>

In [ ]:
from model import DispNet
from dataset import Scene_Flow_disparity

Download Scene Flow Datasets <br>
Make data_paths list for loading data images

In [ ]:
dataset = Scene_Flow_disparity()

Check data method in mode = left, groundTruth mode = disparity

In [ ]:
print(len(dataset.data_paths))
print()
iteration = 0
for dir_ in dataset.data_paths:
    iteration += 1
    if iteration == 200:
        break
    if dataset.data(dir_) is not None:
        print(dir_)
    else:
        print('no left or png path')

# DispNet Simple training<br>

gpu device setting

In [ ]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"

make DispNet simple structure <br>
set training environment(input shape, learning rate, DispNet structure mode, gpu device set)

In [ ]:
import time
import datetime

start=time.clock()

model = DispNet(
    img_height = 540,
    img_width = 960, 
    img_depth = 3, 
    learning_rate = 0.1**4)

net = model.inference('simple')

## training

make training batch input
- train_left_x_batch numpy array
- train_right_x_batch numpy array
<br>

make training batch output
- train_y_batch numpy array
training
- data split in iteration
- no data augmentation

In [ ]:
import cv2
from random import shuffle
from tensorflow.python.keras.callbacks import CSVLogger
from tensorflow.python.keras.callbacks import Callback

csv_logger = CSVLogger('./checkpoints/DispNetSimple_training_log.csv', append=True, separator=';')
TensorBoard = tf.contrib.keras.callbacks.TensorBoard(
    log_dir="TensorBoard/DispNetTensorBoard/DispNetSimple",
    histogram_freq = 0,
    write_graph=True, 
    write_images=True)
class WeightsSaver(Callback):
    def __init__(self, N):
        self.N = N
        self.epoch = 0

    def on_epoch_end(self, epoch, logs={}):
        if self.epoch % self.N == 0:
            self.model.save_weights('./checkpoints/dispnetSimple(%dth).hdf5' % self.epoch)
        self.epoch += 1

def trainDataGenerator(data_paths, target_input_size = (model.model_in_width, model.model_in_height), target_output_size = (model.model_out_height, model.model_out_width), batch_size = 1, suffle = True):
    if shuffle == True:
        shuffle(data_paths)
    train_left_inputBatch = []
    train_right_inputBatch = []
    train_left_groundTruthBatch = []
    train_left_groundTruthBatch2 = []
    train_left_groundTruthBatch3 = []
    train_left_groundTruthBatch4 = []
    train_left_groundTruthBatch5 = []
    train_left_groundTruthBatch6 = []
    for dir_ in data_paths:
        tmp = cv2.imread(dir_, cv2.IMREAD_COLOR)
        l_img_nparray = cv2.resize(tmp, target_input_size, interpolation=cv2.INTER_CUBIC)

        tmp = cv2.imread(dir_.replace("left", "right"), cv2.IMREAD_COLOR)
        r_img_nparray = cv2.resize(tmp, target_input_size, interpolation=cv2.INTER_CUBIC)
        
        if 'driving__frames_cleanpass' in dir_:
            tmp = dir_.replace("driving__frames_cleanpass", "driving__disparity")
            dir_ = tmp.replace("frames_cleanpass", "disparity")
        elif 'driving__frames_finalpass' in dir_:
            tmp = dir_.replace("driving__frames_finalpass", "driving__disparity")
            dir_ = tmp.replace("frames_finalpass", "disparity")
        elif 'flyingthings3d__frames_cleanpass' in dir_:
            tmp = dir_.replace("flyingthings3d__frames_cleanpass", "flyingthings3d__disparity")
            dir_ = tmp.replace("frames_cleanpass", "disparity")
        elif 'flyingthings3d__frames_finalpass' in dir_:
            tmp = dir_.replace("flyingthings3d__frames_finalpass", "flyingthings3d__disparity")
            dir_ = tmp.replace("frames_finalpass", "disparity")
        elif 'monkaa__frames_cleanpass' in dir_:
            tmp = dir_.replace("monkaa__frames_cleanpass", "monkaa__disparity")
            dir_ = tmp.replace("frames_cleanpass", "disparity")
        elif 'monkaa__frames_finalpass' in dir_:
            tmp = dir_.replace("monkaa__frames_finalpass", "monkaa__disparity")
            dir_ = tmp.replace("frames_finalpass", "disparity")
        
        ground_truth = cv2.resize(dataset.read_pfm(dir_.replace(".png", ".pfm")), (model.model_out_width, model.model_out_height), interpolation=cv2.INTER_CUBIC)
        ground_truth2 = cv2.resize(dataset.read_pfm(dir_.replace(".png", ".pfm")), (int(model.model_out_width/2), int(model.model_out_height/2)), interpolation=cv2.INTER_CUBIC)
        ground_truth3 = cv2.resize(dataset.read_pfm(dir_.replace(".png", ".pfm")), (int(model.model_out_width/4), int(model.model_out_height/4)), interpolation=cv2.INTER_CUBIC)
        ground_truth4 = cv2.resize(dataset.read_pfm(dir_.replace(".png", ".pfm")), (int(model.model_out_width/8), int(model.model_out_height/8)), interpolation=cv2.INTER_CUBIC)
        ground_truth5 = cv2.resize(dataset.read_pfm(dir_.replace(".png", ".pfm")), (int(model.model_out_width/16), int(model.model_out_height/16)), interpolation=cv2.INTER_CUBIC)
        ground_truth6 = cv2.resize(dataset.read_pfm(dir_.replace(".png", ".pfm")), (int(model.model_out_width/32), int(model.model_out_height/32)), interpolation=cv2.INTER_CUBIC)
        
        train_left_inputBatch += [l_img_nparray]
        train_right_inputBatch += [r_img_nparray]
        train_left_groundTruthBatch += [ground_truth[:,:,np.newaxis]]
        train_left_groundTruthBatch2 += [ground_truth2[:,:,np.newaxis]]
        train_left_groundTruthBatch3 += [ground_truth3[:,:,np.newaxis]]
        train_left_groundTruthBatch4 += [ground_truth4[:,:,np.newaxis]]
        train_left_groundTruthBatch5 += [ground_truth5[:,:,np.newaxis]]
        train_left_groundTruthBatch6 += [ground_truth6[:,:,np.newaxis]]

        if len(train_left_groundTruthBatch) == batch_size:
            batch_input_x = [np.array(train_left_inputBatch), np.array(train_right_inputBatch)]
            batch_input_y = [np.array(train_left_groundTruthBatch), np.array(train_left_groundTruthBatch2), 
                             np.array(train_left_groundTruthBatch3), np.array(train_left_groundTruthBatch4), 
                             np.array(train_left_groundTruthBatch5), np.array(train_left_groundTruthBatch6)]
            train_left_inputBatch = []
            train_right_inputBatch = []
            train_left_groundTruthBatch = []
            train_left_groundTruthBatch2 = []
            train_left_groundTruthBatch3 = []
            train_left_groundTruthBatch4 = []
            train_left_groundTruthBatch5 = []
            train_left_groundTruthBatch6 = []

            yield (batch_input_x, batch_input_y)

directories = [i for i in dataset.data_paths if ('left' in i) and ('.png' in i)]
shuffle(directories)

train_paths = directories[0:int(len(directories)*0.8)]
train_generator = trainDataGenerator(data_paths = train_paths, batch_size = 2)

validation_paths = directories[int(len(directories)*0.8):]
validation_generator = trainDataGenerator(data_paths = validation_paths, batch_size = 2)

net.fit_generator(train_generator,
                  epochs = 100,
                  steps_per_epoch = int(len(train_paths)/8),
                  validation_data = validation_generator,
                  validation_steps = int(len(validation_paths)/8),
                  callbacks=[TensorBoard, csv_logger, WeightsSaver(10)])

training time check
- day
- hours
- minute
- sec

In [ ]:
runtime_sec = time.clock() - start
m, s = divmod(runtime_sec, 60)
h, m = divmod(m, 60)
d, h = divmod(h, 24)
print('runtime : %d days %d:%02d:%02d' % (d, h, m, s))

save trained model weight

In [ ]:
net.save_weights('./checkpoints/dispnetSimple.hdf5')

# DispNet Correlation training<br>

gpu device setting

In [ ]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"

make DispNet simple structure <br>
set training environment(input shape, learning rate, DispNet structure mode)

In [ ]:
import time
import datetime

model = DispNet(
    img_height = 540, 
    img_width = 960, 
    img_depth = 3, 
    learning_rate = 0.1**4)

net = model.inference('correlation')

start=time.clock()

## training

make training batch input
- train_left_x_batch numpy array
- train_right_x_batch numpy array
<br>

make training batch output
- train_y_batch numpy array
training
- data split in iteration
- no data augmentation

In [ ]:
import cv2
from random import shuffle
from tensorflow.python.keras.callbacks import CSVLogger
from tensorflow.python.keras.callbacks import Callback

csv_logger = CSVLogger('./checkpoints/DispNetCorr_training_log.csv', append=True, separator=';')
TensorBoard = tf.contrib.keras.callbacks.TensorBoard(
    log_dir="TensorBoard/DispNetTensorBoard/DispNetCorr",
    histogram_freq = 0,
    write_graph=True, 
    write_images=True)
class WeightsSaver(Callback):
    def __init__(self, N):
        self.N = N
        self.epoch = 0

    def on_epoch_end(self, epoch, logs={}):
        if self.epoch % self.N == 0:
            self.model.save_weights('./checkpoints/dispnetCorr(%dth).hdf5' % self.epoch)
        self.epoch += 1

def trainDataGenerator(data_paths, target_input_size = (model.model_in_width, model.model_in_height), target_output_size = (model.model_out_height, model.model_out_width), batch_size = 1, suffle = True):
    if shuffle == True:
        shuffle(data_paths)
    train_left_inputBatch = []
    train_right_inputBatch = []
    train_left_groundTruthBatch = []
    train_left_groundTruthBatch2 = []
    train_left_groundTruthBatch3 = []
    train_left_groundTruthBatch4 = []
    train_left_groundTruthBatch5 = []
    train_left_groundTruthBatch6 = []
    for dir_ in data_paths:
        tmp = cv2.imread(dir_, cv2.IMREAD_COLOR)
        l_img_nparray = cv2.resize(tmp, target_input_size, interpolation=cv2.INTER_CUBIC)

        tmp = cv2.imread(dir_.replace("left", "right"), cv2.IMREAD_COLOR)
        r_img_nparray = cv2.resize(tmp, target_input_size, interpolation=cv2.INTER_CUBIC)
        
        if 'driving__frames_cleanpass' in dir_:
            tmp = dir_.replace("driving__frames_cleanpass", "driving__disparity")
            dir_ = tmp.replace("frames_cleanpass", "disparity")
        elif 'driving__frames_finalpass' in dir_:
            tmp = dir_.replace("driving__frames_finalpass", "driving__disparity")
            dir_ = tmp.replace("frames_finalpass", "disparity")
        elif 'flyingthings3d__frames_cleanpass' in dir_:
            tmp = dir_.replace("flyingthings3d__frames_cleanpass", "flyingthings3d__disparity")
            dir_ = tmp.replace("frames_cleanpass", "disparity")
        elif 'flyingthings3d__frames_finalpass' in dir_:
            tmp = dir_.replace("flyingthings3d__frames_finalpass", "flyingthings3d__disparity")
            dir_ = tmp.replace("frames_finalpass", "disparity")
        elif 'monkaa__frames_cleanpass' in dir_:
            tmp = dir_.replace("monkaa__frames_cleanpass", "monkaa__disparity")
            dir_ = tmp.replace("frames_cleanpass", "disparity")
        elif 'monkaa__frames_finalpass' in dir_:
            tmp = dir_.replace("monkaa__frames_finalpass", "monkaa__disparity")
            dir_ = tmp.replace("frames_finalpass", "disparity")
        
        ground_truth = cv2.resize(dataset.read_pfm(dir_.replace(".png", ".pfm")), (model.model_out_width, model.model_out_height), interpolation=cv2.INTER_CUBIC)
        ground_truth2 = cv2.resize(dataset.read_pfm(dir_.replace(".png", ".pfm")), (int(model.model_out_width/2), int(model.model_out_height/2)), interpolation=cv2.INTER_CUBIC)
        ground_truth3 = cv2.resize(dataset.read_pfm(dir_.replace(".png", ".pfm")), (int(model.model_out_width/4), int(model.model_out_height/4)), interpolation=cv2.INTER_CUBIC)
        ground_truth4 = cv2.resize(dataset.read_pfm(dir_.replace(".png", ".pfm")), (int(model.model_out_width/8), int(model.model_out_height/8)), interpolation=cv2.INTER_CUBIC)
        ground_truth5 = cv2.resize(dataset.read_pfm(dir_.replace(".png", ".pfm")), (int(model.model_out_width/16), int(model.model_out_height/16)), interpolation=cv2.INTER_CUBIC)
        ground_truth6 = cv2.resize(dataset.read_pfm(dir_.replace(".png", ".pfm")), (int(model.model_out_width/32), int(model.model_out_height/32)), interpolation=cv2.INTER_CUBIC)
        
        train_left_inputBatch += [l_img_nparray]
        train_right_inputBatch += [r_img_nparray]
        train_left_groundTruthBatch += [ground_truth[:,:,np.newaxis]]
        train_left_groundTruthBatch2 += [ground_truth2[:,:,np.newaxis]]
        train_left_groundTruthBatch3 += [ground_truth3[:,:,np.newaxis]]
        train_left_groundTruthBatch4 += [ground_truth4[:,:,np.newaxis]]
        train_left_groundTruthBatch5 += [ground_truth5[:,:,np.newaxis]]
        train_left_groundTruthBatch6 += [ground_truth6[:,:,np.newaxis]]

        if len(train_left_groundTruthBatch) == batch_size:
            batch_input_x = [np.array(train_left_inputBatch), np.array(train_right_inputBatch)]
            batch_input_y = [np.array(train_left_groundTruthBatch), np.array(train_left_groundTruthBatch2), 
                             np.array(train_left_groundTruthBatch3), np.array(train_left_groundTruthBatch4), 
                             np.array(train_left_groundTruthBatch5), np.array(train_left_groundTruthBatch6)]
            train_left_inputBatch = []
            train_right_inputBatch = []
            train_left_groundTruthBatch = []
            train_left_groundTruthBatch2 = []
            train_left_groundTruthBatch3 = []
            train_left_groundTruthBatch4 = []
            train_left_groundTruthBatch5 = []
            train_left_groundTruthBatch6 = []

            yield (batch_input_x, batch_input_y)

directories = [i for i in dataset.data_paths if ('left' in i) and ('.png' in i)]
shuffle(directories)

train_paths = directories[0:int(len(directories)*0.8)]
train_generator = trainDataGenerator(data_paths = train_paths, batch_size = 2)

validation_paths = directories[int(len(directories)*0.8):]
validation_generator = trainDataGenerator(data_paths = validation_paths, batch_size = 2)

net.fit_generator(train_generator,
                  epochs = 100,
                  steps_per_epoch = int(len(train_paths)/8),
                  validation_data = validation_generator,
                  validation_steps = int(len(validation_paths)/8),
                  callbacks=[TensorBoard, csv_logger, WeightsSaver(10)])

training time check
- day
- hours
- minute
- sec

In [ ]:
runtime_sec = time.clock() - start
m, s = divmod(runtime_sec, 60)
h, m = divmod(m, 60)
d, h = divmod(h, 24)
print('runtime : %d days %d:%02d:%02d' % (d, h, m, s))

save trained model weight

In [ ]:
net.save_weights('./checkpoints/dispnetCorr.hdf5')